In [4]:
import pandas as pd
import re
import tkinter as tk
from datetime import datetime
from tkinter import filedialog, messagebox, ttk
import time
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
import threading
import os
import shutil

# Keywords for NACE classification
nace_keywords = {
    "A1": [
        "agriculture", "crop", "palm oil", "rice", "cotton", "cocoa", "coffee", "sugar",
        "landwirtschaft", "ackerbau", "palmenöl", "reis", "baumwolle", "kakao", "kaffee", "zucker"
    ],
    "A2": [
        "forestry", "wood", "logging", "timber",
        "forstwirtschaft", "holz", "abholzung", "forst", "holzernte"
    ],
    "A3": [
        "fishing", "aquaculture", "fish", "shrimp",
        "fischerei", "aquakultur", "fisch", "garnelen"
    ],
    "C10": [
        "cocoa", "chocolate", "coffee", "tea", "sugar", "food",
        "kakao", "schokolade", "kaffee", "tee", "zucker", "lebensmittel", "nahrungsmittel"
    ],
    "C13": [
        "textile", "fabric", "yarn", "garment",
        "textilien", "stoff", "garn", "gewebe", "textil"
    ],
    "C14": [
        "clothing", "apparel", "garment", "t-shirt", "shirt", "jeans",
        "kleidung", "bekleidung", "oberbekleidung", "t-shirt", "hemd", "jeans"
    ],
    "C15": [
        "leather", "shoe", "footwear",
        "leder", "schuh", "schuhe", "fußbekleidung"
    ],
    "C20": [
        "chemical", "pharmaceutical", "fertilizer", "pesticide", "plastic",
        "chemie", "chemikalien", "dünger", "pestizid", "kunststoff"
    ],
    "C21": [
        "pharma", "pharmaceutical", "medicine",
        "pharma", "pharmazeutisch", "medizin", "arznei", "medikament"
    ],
    "C22": [
        "plastic product", "plastic",
        "kunststoff", "plastik", "kunststoffprodukt"
    ],
    "C26": [
        "electronics", "electronic", "semiconductor", "battery",
        "elektronik", "elektronisch", "halbleiter", "batterie"
    ],
    "D35": [
        "energy", "electricity", "power",
        "energie", "strom", "elektrizität"
    ],
    "E38": [
        "waste", "recycling", "disposal",
        "abfall", "recycling", "entsorgung"
    ],
    "E39": [
        "remediation", "cleanup",
        "sanierung", "reinigung", "aufbereitung"
    ],
    "H49": [
        "transport", "freight", "truck", "rail",
        "transport", "fracht", "lkw", "bahn", "schiene"
    ],
    "H50": [
        "shipping", "sea freight",
        "schifffahrt", "seehandel", "seefracht"
    ],
    "H52": [
        "warehouse", "storage", "logistics",
        "lager", "lagerung", "logistik", "lagerhaltung"
    ],
    "G46": [
        "wholesale", "trade",
        "großhandel", "handel"
    ],
    "G47": [
        "retail", "store", "shop",
        "einzelhandel", "laden", "geschäft"
    ]
}

# Trustworthiness scoring
def score_sources(source_name, author_name, language, date_str, sector_relevant, has_sources, independent):
    score = 0

    # 1. Source and authority of the information

    trusted_high = ["UN", "OECD", "ILO", "EU", "Bundesministerium", "Federal Ministry", "University", "Max Planck",
    "UNEP", "UNICEF", "WHO", "World Bank", "Weltbank", "BMZ", "BAFA", "GIZ",
    "Fraunhofer", "Helmholtz", "Amnesty", "Greenpeace", "FAO", "UNDP"]
    trusted_medium = ["NGO", "Fairtrade", "McKinsey", "Oxfam", "Vision",
    "AidEnvironment", "Foodwatch", "Transparency International", "WWF", "Brot für die Welt",
    "Verité", "World Vision", "Bridge Michigan"]
    source = str(source_name)
    if any(key.lower() in source.lower() for key in trusted_high):
        score += 10
    elif any(key.lower() in source.lower() for key in trusted_medium):
        score += 7
    else:
        score += 4

    # 2. Actuality
    try:
        date = datetime.strptime(date_str, "%Y-%m-%d")
        tage_alt = (datetime.today() - date).days
        if tage_alt < 365:
            score += 10
        elif tage_alt < 1095:
            score += 8
        elif tage_alt < 2190:
            score += 6
        else:
            score += 3
    except:
        score += 5  # Fallback

    # 3. Author competence
    competent_authors = ["Dr.", "Prof.", "Expert", "Researcher", "PhD", "M.Sc.", "Msc", "Dipl.-Ing.", "Diplom", "Mag.", "Mba"]
    if any(k.lower() in author_name.lower() for k in competent_authors):
        score += 10
    else:
        score += 6

    # 4. Transparency
    score += 10 if has_sources else 5

    # 5. Language
    if language.lower() == "english":
        score += 10
    elif language.lower() == "deutsch" or language.lower() == "german":
        score += 7
    else:
        score += 5

    # 6. Sectory relevance
    score += 10 if sector_relevant else 5

    # 7. Independency
    score += 10 if independent else 5

    # Final score 
    final_score = round(score / 7, 1)  # average score of all 7 criteria
    return final_score

def score_risk(severity, probability, scope, urgency, nace_relevant, company_influence, data_availability, regulation):

    # Scoring scales
    scale = {"HIGH": 10, "MEDIUM": 7, "LOW": 4}
    scope_scale = {"GLOBAL": 10, "REGIONAL": 7, "LOCAL": 4}
    urgency_scale = {"IMMEDIATE": 10, "MEDIUM TERM": 7, "LONG TERM": 4}

    # Highest possible score: 8 × 10 points = 80
    score = 0
    score += scale.get(severity, 5)
    score += scale.get(probability, 5)
    score += scope_scale.get(scope, 5)
    score += urgency_scale.get(urgency, 5)
    score += scale.get(nace_relevant, 5)
    score += scale.get(company_influence, 5)
    score += scale.get(data_availability, 5)
    score += scale.get(regulation, 5)

    # 1-10 scale for final score
    final_score = round((score / 80) * 10, 1)
    return final_score


# Check if entry is already shown in the GUI
def entry_already_in_gui(title, author):
    for child in tree.get_children():
        values = tree.item(child)["values"]
        if len(values) >= 2 and values[0] == title and values[1] == author:
            return True
    return False

# Analyze input Excel file
def analyze_file(path):
    try:
        df_input = pd.read_excel(path)
    except Exception as e:
        messagebox.showerror("Error loading file", str(e))
        return

    filename = "nace_source_evaluation.xlsx"
    if os.path.exists(filename):
        df_existing = pd.read_excel(filename)
    else:
        df_existing = pd.DataFrame()

    results = []
    skipped_titles = []

    for _, row in df_input.iterrows():
        title = str(row["Title"])
        author = str(row["Author"])

        if entry_already_in_gui(title, author):
            skipped_titles.append(title)
            continue

        # Assign NACE codes
        matched_codes = []
        for code, keywords in nace_keywords.items():
            if any(re.search(rf"\b{kw}\b", title, re.IGNORECASE) for kw in keywords):
                matched_codes.append(code)
        nace_codes = ", ".join(matched_codes) if matched_codes else "Undefined"

        timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
        source = str(row["Source"])
        language = str(row["Language"])
        date_str = str(row["Date"])
        sector_relevant = bool(row["Sector_Relevant"])
        has_sources = bool(row["Has_Sources"])
        independent = bool(row["Independent"])

        trust = score_sources(
    source, author, language, date_str,
    sector_relevant, has_sources, independent
)
        
        severity = str(row["Severity"])
        probability = str(row["Probability"])
        scope = str(row["Scope"])
        urgency = str(row["Urgency"])
        nace_relevant = str(row["NACE_Relevant"])
        company_influence = str(row["Company_Influence"])
        data_availability = str(row["Data_Availability"])
        regulation = str(row["Regulation"])

        risk = score_risk(severity, probability, scope, urgency,
                  nace_relevant, company_influence, data_availability, regulation)
        
        results.append({
            "Title": title,
            "Author": author,
            "NACE_Assignment": nace_codes,
            "Trust_Score": trust,
            "Risk_Score": risk,
            "Timestamp": timestamp
        })

    if skipped_titles:
        info_text = "The following titles have already been analyzed and were skipped:\n\n" + "\n".join(skipped_titles)
        messagebox.showinfo("Info", info_text)

    if not results:
        return

    df_output = pd.DataFrame(results)

    if not df_existing.empty:
        df_total = pd.concat([df_existing, df_output], ignore_index=True)
    else:
        df_total = df_output

    df_total.to_excel(filename, index=False)
    return df_output

# Load and process file from GUI
def load_file():
    path = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx")])
    if not path:
        return

    df = analyze_file(path)
    if df is not None:
        for _, row in df.iterrows():
            title = row["Title"]
            author = row["Author"]
            if not entry_already_in_gui(title, author):
                tree.insert("", tk.END, values=list(row))

        processed_folder = "processed"
        os.makedirs(processed_folder, exist_ok=True)

        filename = os.path.basename(path)
        target_path = os.path.join(processed_folder, filename)

        try:
            shutil.move(path, target_path)
        except Exception as e:
            print(f"Error moving file: {e}")

        messagebox.showinfo("Done", "Analysis completed and saved as 'nace_source_evaluation.xlsx'")

# GUI setup
root = tk.Tk()
root.title("NACE Source Analysis Tool")
root.geometry("1200x600")
root.configure(bg="#f0f4f8")

frame_top = tk.Frame(root, bg="#f0f4f8")
frame_top.pack(pady=20)

lbl_title = tk.Label(frame_top, text="NACE Classification and Trust Evaluation", font=("Arial", 16, "bold"), bg="#f0f4f8")
lbl_title.pack(pady=5)

btn_load = tk.Button(frame_top, text="Load & Analyze Excel File", command=load_file, font=("Arial", 12), bg="#4CAF50", fg="white", padx=10, pady=5)
btn_load.pack()

# --- INPUT ---
input_fields = [
    "Title", "Source", "Author", "Language", "Date", "Sector_Relevant",
    "Has_Sources", "Independent", "Severity", "Probability", "Scope",
    "Urgency", "NACE_Relevant", "Company_Influence", "Data_Availability", "Regulation"
]

entry_frame = tk.Frame(root, bg="#f0f4f8")
entry_frame.pack(fill="x", padx=10, pady=10)

canvas = tk.Canvas(entry_frame, height=50, bg="#f0f4f8")
scrollbar = tk.Scrollbar(entry_frame, orient="horizontal", command=canvas.xview)
scrollable_input_frame = tk.Frame(canvas, bg="#f0f4f8")

scrollable_input_frame.bind(
    "<Configure>", lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
)
canvas.create_window((0, 0), window=scrollable_input_frame, anchor="nw")
canvas.configure(xscrollcommand=scrollbar.set)

canvas.pack(side="top", fill="x", expand=True)
scrollbar.pack(side="bottom", fill="x")

entry_widgets = {}

for idx, field in enumerate(input_fields):
    tk.Label(scrollable_input_frame, text=f"{field}:", font=("Arial", 12), bg="#f0f4f8")\
        .grid(row=0, column=idx, padx=5, sticky="w")
    entry = tk.Entry(scrollable_input_frame, width=20, font=("Arial", 12))
    entry.grid(row=1, column=idx, padx=5, pady=5)
    entry_widgets[field] = entry

def add_manual_entry():
    values = {k: entry_widgets[k].get().strip() for k in input_fields}
    
    # Title and Source are mandatory
    if not values["Title"] or not values["Source"]:
        messagebox.showwarning("Input Error", "Please enter both Title and Source.")
        return

    filename = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx")])
    if not filename:
        return

    try:
        df_existing = pd.read_excel(filename)
    except Exception as e:
        messagebox.showerror("Error reading file", str(e))
        return

    if ((df_existing["Title"] == values["Title"]) & (df_existing["Author"] == values["Source"])).any():
        messagebox.showinfo("Duplicate", "This entry already exists in the file.")
        return

    # NACE-Zuordnung
    matched_codes = []
    for code, keywords in nace_keywords.items():
        if any(re.search(rf"\b{kw}\b", values["Title"], re.IGNORECASE) for kw in keywords):
            matched_codes.append(code)
    nace_codes = ", ".join(matched_codes) if matched_codes else "Undefined"

    # Vertrauens-Score berechnen
    trust = score_sources(
        values["Source"],
        values.get("Author", ""),
        values.get("Language", ""),
        values.get("Date", ""),
        values.get("Sector_Relevant", "").lower() == "true",
        values.get("Has_Sources", "").lower() == "true",
        values.get("Independent", "").lower() == "true"
    )

    # Risiko-Score berechnen
    risk = score_risk(
        values.get("Severity", ""), values.get("Probability", ""), values.get("Scope", ""),
        values.get("Urgency", ""), values.get("NACE_Relevant", ""), values.get("Company_Influence", ""),
        values.get("Data_Availability", ""), values.get("Regulation", "")
    )

    timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
    author = values.get("Author") or values["Source"]

    new_row = {
        "Title": values["Title"],
        #"Source": values["Source"],
        "Author": author,
        "NACE_Assignment": nace_codes,
        "Trust_Score": trust,
        "Risk_Score": risk,
        "Timestamp": timestamp
    }

    df_updated = pd.concat([df_existing, pd.DataFrame([new_row])], ignore_index=True)

    try:
        df_updated.to_excel(filename, index=False)
    except Exception as e:
        messagebox.showerror("Error writing to file", str(e))
        return

    if not entry_already_in_gui(values["Title"], values["Source"]):
        tree.insert("", tk.END, values=list(new_row.values()))

    messagebox.showinfo("Success", "Entry added successfully.")

# Button
btn_add_manual = tk.Button(entry_frame, text="Add Entry to File", command=add_manual_entry,
                           font=("Arial", 12), bg="#2196F3", fg="white", padx=10, pady=5)
btn_add_manual.pack(pady=10)
# --- INPUT ---

# --- STATIC LEGEND ---
legend_frame = tk.Frame(root, bg="#f0f4f8", pady=10)
legend_frame.pack(fill="x", padx=10)

# Trust Score Legende
trust_label = tk.Label(
    legend_frame,
    text="Trust Score: 10 = Trusted high, 7 = Trusted medium 4 = Not trusted",
    font=("Arial", 11),
    bg="#f0f4f8",
    fg="#333"
)
trust_label.pack(anchor="w", padx=5)

# Risk Score Legende
risk_label = tk.Label(
    legend_frame,
    text="Risk Score: 10 = Hohes Risiko, 1 = Geringes Risiko",
    font=("Arial", 11),
    bg="#f0f4f8",
    fg="#333"
)
risk_label.pack(anchor="w", padx=5)
# --- END LEGEND ---

frame_table = tk.Frame(root)
frame_table.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

columns = ["Title", "Author", "NACE_Assignment", "Trust_Score", "Risk_Score", "Timestamp"]
tree = ttk.Treeview(frame_table, columns=columns, show="headings", height=20)

style = ttk.Style()
style.configure("Treeview.Heading", font=("Arial", 11, "bold"))
style.configure("Treeview", font=("Arial", 10))

for col in columns:
    tree.heading(col, text=col)
    tree.column(col, anchor="w", width=230)
tree.pack(fill=tk.BOTH, expand=True)

# Watchdog handler
class ExcelHandler(FileSystemEventHandler):
    def on_created(self, event):
        if event.src_path.endswith(".xlsx"):
            df = analyze_file(event.src_path)
            if df is not None:
                tree.after(0, update_gui, df)
                processed_folder = "processed"
                os.makedirs(processed_folder, exist_ok=True)
                filename = os.path.basename(event.src_path)
                target_path = os.path.join(processed_folder, filename)
                try:
                    shutil.move(event.src_path, target_path)
                except Exception as e:
                    print(f"Error moving file: {e}")

# Update GUI with new data
def update_gui(df):
    for _, row in df.iterrows():
        title = row["Title"]
        author = row["Author"]
        if not entry_already_in_gui(title, author):
            tree.insert("", tk.END, values=list(row))

# Folder monitoring
def monitor_folder(path):
    os.makedirs(path, exist_ok=True)
    event_handler = ExcelHandler()
    observer = Observer()
    observer.schedule(event_handler, path=path, recursive=False)
    observer.start()

monitor_thread = threading.Thread(
    target=monitor_folder, 
    args=("incoming_sources",), 
    daemon=True
)
monitor_thread.start()

root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\domin\anaconda3\envs\DataChallenge\Lib\site-packages\pandas\core\indexes\base.py", line 3653, in get_loc
    return self._engine.get_loc(casted_key)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "pandas\_libs\index.pyx", line 147, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 176, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 7080, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 7088, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'Source'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\domin\anaconda3\envs\DataChallenge\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\domin\AppData\Local\Temp\ipy